In [1]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7f72ad4dc398; dsn: 'dbname=postgresdemo user=clientdemo password=xxxxxxxxxxx host=postgresdemo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [4]:
doc['column_map']

{'CUSTOMER_CONTACT': {'cust_id': 'id',
  'email_id': 'email',
  'facebook_id': 'facebook_id',
  'firstname': 'first_name',
  'lastname': 'last_name',
  'phone_number': 'phone'},
 'CUSTOMER_MASTER': {'birthday': 'birthday',
  'create_date': 'create_date',
  'cust_id': 'id'},
 'CUSTOMER_SECONDARY': {'cust_id': 'cust_id'},
 'EVENT_LOG': {'action_type': 'actiontype',
  'cust_id': 'from_id',
  'product_id': 'to_id',
  'timestamp': 'timestamp'},
 'EVENT_MASTER': {'action_category_id': None,
  'action_category_name': None,
  'action_name': 'action_desc',
  'action_type': 'actiontype'},
 'TRANSACTION_MASTER': {'cust_id': 'userid',
  'product_id': 'product_id',
  'quantity': 'quantity',
  'renewal': 'renewal',
  'revenue': 'revenue',
  'timestamp': 'purchase_date'}}

In [5]:
query = 'SELECT "%s","%s","%s" FROM %s'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                                         doc['column_map']['TRANSACTION_MASTER']['product_id'],
                                         doc['column_map']['TRANSACTION_MASTER']['timestamp'],
                                         doc['table_map']['TRANSACTION_MASTER'])

In [6]:
print query

SELECT "userid","product_id","purchase_date" FROM trans


In [7]:
temp.conn.rollback()

In [8]:
temp.cur.execute(query)

In [9]:
col_names = [desc[0] for desc in temp.cur.description]

In [10]:
col_names

['userid', 'product_id', 'purchase_date']

In [11]:
import pandas as pd

In [12]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [13]:
trans_table.head()

,userid,product_id,purchase_date
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00


In [14]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id'] : 'cust_id'}, inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id'] : 'product_id'}, inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp'] : 'timestamp'}, inplace=True)

In [15]:
trans_table.head()

,cust_id,product_id,timestamp
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00


In [16]:
trans_table.shape

(4428, 3)

In [17]:
len(trans_table['cust_id'].unique())

3087

In [18]:
cust_prod_comb = trans_table.groupby(['cust_id', 'product_id']).size().reset_index().rename(columns = {0:'count'})

In [19]:
cust_prod_comb.head()

,cust_id,product_id,count
0,2942914,com.gopaktor.subscription.v1.premium.1m,1
1,3053432,com.gopaktor.subscription.v4.premium.1m,10
2,3129800,com.gopaktor.subscription.v1.premium.1m,1
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,1
4,3325662,com.gopaktor.subscription.v12.premium.1m,2


In [20]:
cust_prod_comb.shape

(3232, 3)

In [21]:
cust_date_comb = trans_table[['cust_id', 'timestamp']]

In [22]:
def x(frame):
    frame.sort_values('timestamp', inplace=True)
    frame['repurchase_time'] = frame['timestamp'] - frame['timestamp'].shift(1)
    return frame

In [44]:
dates = trans_table.groupby(['cust_id', 'product_id'], group_keys=False).apply(x)

In [45]:
dates.head()

,cust_id,product_id,timestamp,repurchase_time
2790,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaT
4309,3053432,com.gopaktor.subscription.v4.premium.1m,2016-05-26 04:27:04+00:00,NaT
4308,3053432,com.gopaktor.subscription.v4.premium.1m,2016-06-27 06:17:00+00:00,32 days 01:49:56
3554,3053432,com.gopaktor.subscription.v4.premium.1m,2016-07-27 06:17:00+00:00,30 days 00:00:00
3553,3053432,com.gopaktor.subscription.v4.premium.1m,2016-08-27 06:17:00+00:00,31 days 00:00:00


In [46]:
dates.sort_index(inplace=True)

In [47]:
dates.head()

,cust_id,product_id,timestamp,repurchase_time
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00,NaT
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00,NaT
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00,NaT
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00,NaT
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00,NaT


In [52]:
dates['sum'] = dates['repurchase_time'].groupby(dates['cust_id']).transform('sum')

In [53]:
dates.head()

,cust_id,product_id,timestamp,repurchase_time,sum
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00,NaT,91 days 01:00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00,NaT,91 days 01:00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00,NaT,NaT
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00,NaT,NaT
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00,NaT,NaT


In [54]:
dates.query('cust_id == "3053432"')

,cust_id,product_id,timestamp,repurchase_time,sum
3553,3053432,com.gopaktor.subscription.v4.premium.1m,2016-08-27 06:17:00+00:00,31 days 00:00:00,245 days 01:54:56
3554,3053432,com.gopaktor.subscription.v4.premium.1m,2016-07-27 06:17:00+00:00,30 days 00:00:00,245 days 01:54:56
3555,3053432,com.gopaktor.subscription.v4.premium.1m,2016-09-27 06:18:00+00:00,0 days 00:01:00,245 days 01:54:56
3556,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,31 days 00:00:00,245 days 01:54:56
4308,3053432,com.gopaktor.subscription.v4.premium.1m,2016-06-27 06:17:00+00:00,32 days 01:49:56,245 days 01:54:56
4309,3053432,com.gopaktor.subscription.v4.premium.1m,2016-05-26 04:27:04+00:00,NaT,245 days 01:54:56
4310,3053432,com.gopaktor.subscription.v4.premium.1m,2016-09-27 06:17:00+00:00,31 days 00:00:00,245 days 01:54:56
4311,3053432,com.gopaktor.subscription.v4.premium.1m,2016-10-27 06:17:00+00:00,29 days 23:59:00,245 days 01:54:56
4312,3053432,com.gopaktor.subscription.v4.premium.1m,2016-11-26 06:22:00+00:00,30 days 00:05:00,245 days 01:54:56
4313,3053432,com.gopaktor.subscription.v4.premium.1m,2016-12-26 06:22:00+00:00,30 days 00:00:00,245 days 01:54:56


In [55]:
dates.head()

,cust_id,product_id,timestamp,repurchase_time,sum
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00,NaT,91 days 01:00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00,NaT,91 days 01:00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00,NaT,NaT
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00,NaT,NaT
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00,NaT,NaT


In [56]:
cust_prod_comb.head()

,cust_id,product_id,count
0,2942914,com.gopaktor.subscription.v1.premium.1m,1
1,3053432,com.gopaktor.subscription.v4.premium.1m,10
2,3129800,com.gopaktor.subscription.v1.premium.1m,1
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,1
4,3325662,com.gopaktor.subscription.v12.premium.1m,2


In [73]:
final = pd.merge(cust_prod_comb, dates, on=['cust_id','product_id'])

In [74]:
final.head(20)

,cust_id,product_id,count,timestamp,repurchase_time,sum
0,2942914,com.gopaktor.subscription.v1.premium.1m,1,2016-12-24 09:53:35+00:00,NaT,NaT
1,3053432,com.gopaktor.subscription.v4.premium.1m,10,2016-08-27 06:17:00+00:00,31 days 00:00:00,245 days 01:54:56
2,3053432,com.gopaktor.subscription.v4.premium.1m,10,2016-07-27 06:17:00+00:00,30 days 00:00:00,245 days 01:54:56
3,3053432,com.gopaktor.subscription.v4.premium.1m,10,2016-09-27 06:18:00+00:00,0 days 00:01:00,245 days 01:54:56
4,3053432,com.gopaktor.subscription.v4.premium.1m,10,2017-01-26 06:22:00+00:00,31 days 00:00:00,245 days 01:54:56
5,3053432,com.gopaktor.subscription.v4.premium.1m,10,2016-06-27 06:17:00+00:00,32 days 01:49:56,245 days 01:54:56
6,3053432,com.gopaktor.subscription.v4.premium.1m,10,2016-05-26 04:27:04+00:00,NaT,245 days 01:54:56
7,3053432,com.gopaktor.subscription.v4.premium.1m,10,2016-09-27 06:17:00+00:00,31 days 00:00:00,245 days 01:54:56
8,3053432,com.gopaktor.subscription.v4.premium.1m,10,2016-10-27 06:17:00+00:00,29 days 23:59:00,245 days 01:54:56
9,3053432,com.gopaktor.subscription.v4.premium.1m,10,2016-11-26 06:22:00+00:00,30 days 00:05:00,245 days 01:54:56


In [75]:
del final['timestamp']

In [76]:
del final['repurchase_time']

In [77]:
final.head()

,cust_id,product_id,count,sum
0,2942914,com.gopaktor.subscription.v1.premium.1m,1,NaT
1,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56
2,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56
3,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56
4,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56


In [78]:
final.shape

(4428, 4)

In [81]:
final_data = final

In [82]:
final_data.head()

,cust_id,product_id,count,sum
0,2942914,com.gopaktor.subscription.v1.premium.1m,1,NaT
1,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56
2,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56
3,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56
4,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56


In [83]:
final_data.shape

(4428, 4)

In [84]:
len(final_data.cust_id.unique())

3087

In [86]:
len(final_data.product_id.unique())

18

In [87]:
final_data['repurchase_time'] = final_data['sum'] / final_data['count']

In [88]:
final_data.head()

,cust_id,product_id,count,sum,repurchase_time
0,2942914,com.gopaktor.subscription.v1.premium.1m,1,NaT,NaT
1,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56,24 days 12:11:29.600000
2,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56,24 days 12:11:29.600000
3,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56,24 days 12:11:29.600000
4,3053432,com.gopaktor.subscription.v4.premium.1m,10,245 days 01:54:56,24 days 12:11:29.600000


In [ ]:
final_data['repurchase_time'] = final_data['repurchase_time'].fillna(0)

In [ ]:
final_data.head()

In [ ]:
final_data.rename(columns={'repurchase_time':'customer_repurchase_time'}, inplace=True)

In [ ]:
final_data.head()

In [ ]:
final_data.query('product_id == "com.gopaktor.credit.v2.6000"')